<a href="https://colab.research.google.com/github/edcalderin/course-mlops-zoomcamp/blob/master/homeworks/01_intro_homework.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework
The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

In [93]:
!mkdir data
!python -m wget -o data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet

Ya existe el subdirectorio o el archivo data.



Saved under data/yellow_tripdata_2022-01.parquet


In [2]:
import pandas as pd

In [97]:
data_january = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
data_january.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


Read the data for January. How many columns are there?

* 16
* 17
* 18
* 19

In [5]:
len(data_january.columns)

19

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

* 41.45
* 46.45
* 51.45
* 56.45

In [98]:
def calculate_duration(df:pd.DataFrame):
    duration = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
    return duration.dt.total_seconds().div(60).astype(int)

In [99]:
data_january['duration'] = calculate_duration(data_january)

In [8]:
print(data_january.duration.std().round(2))

46.45


## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

* 90%
* 92%
* 95%
* 98%

In [100]:
data_january['is_outlier'] = (data_january.duration<1) | (data_january.duration>60)
data_january.is_outlier.value_counts(normalize=True)

is_outlier
False    0.98352
True     0.01648
Name: proportion, dtype: float64

In [101]:
data_january = data_january[~data_january.is_outlier]
data_january.shape

(2423325, 21)

## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

* 2
* 155
* 345
* 515
* 715

In [102]:
def convert_to_dicts(df:pd.DataFrame, category_names=['PULocationID', 'DOLocationID']):
    df[category_names] = df[category_names].astype('str')

    return df[category_names].to_dict(orient='records')

In [103]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer()

X_train = dv.fit_transform(convert_to_dicts(data_january))

[fn for fn in dv.feature_names_[:10]]

['DOLocationID=1',
 'DOLocationID=10',
 'DOLocationID=100',
 'DOLocationID=101',
 'DOLocationID=102',
 'DOLocationID=105',
 'DOLocationID=106',
 'DOLocationID=107',
 'DOLocationID=108',
 'DOLocationID=109']

In [41]:
len(dv.feature_names_)

515

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

* Train a plain linear regression model with default parameters
* Calculate the RMSE of the model on the training data  

What's the RMSE on train?

* 6.99
* 11.99
* 16.99
* 21.99

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Model creation
lr_model = LinearRegression()

# Training
y = data_january.duration
lr_model.fit(X_train, y)

# Evaluation

def rmse(model, feature_matrix, y):
    predicted = model.predict(feature_matrix)

    rmse = mean_squared_error(y, predicted, squared=False)
    return rmse

print(rmse(lr_model, X_train, data_january.duration))

7.0167202695956865


## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

* 7.79
* 12.79
* 17.79
* 22.79

In [94]:
!python -m wget -o data https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet


Saved under data/yellow_tripdata_2022-02.parquet


In [105]:
data_february = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

# Calculating duration
data_february['duration'] = calculate_duration(data_february)

# Dropping outliers
data_february = data_february[(data_february.duration>=1) & (data_february.duration<=60)]

# Transforming february's data
X_test = dv.transform(convert_to_dicts(data_february))

# Evaluating
print(rmse(lr_model, X_test, data_february.duration))

7.827338705733996


Answer: 7.79

## Persisting model

In [109]:
!mkdir models

In [111]:
import pickle

FILE_NAME = "models/lin_reg.bin"

with open(FILE_NAME, 'wb') as file:
    pickle.dump((dv, lr_model), file)

In [112]:
# get file size in python
import os

file_stats = os.stat(FILE_NAME)

print(file_stats)
print(f'File Size in KiloBytes is {file_stats.st_size / (1024)}')

os.stat_result(st_mode=33206, st_ino=35184372089190499, st_dev=4272072175, st_nlink=1, st_uid=0, st_gid=0, st_size=17369, st_atime=1684714255, st_mtime=1684714255, st_ctime=1684714255)
File Size in KiloBytes is 16.9619140625
